Imports and endpoints

In [1]:
from functools import lru_cache
import requests
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
from time import sleep

urls = {
    'api' : 'https://api.mtga.untapped.gg/api/v1/',
    'json' : 'https://mtgajson.untapped.gg/v1/latest/',
}

endpoints = {
    'active': ('api', 'meta-periods/active'),
    'analytics': ('api', 'analytics/query/card_stats_by_archetype_event_and_scope_free/ALL?MetaPeriodId='),
    'cards': ('json', 'cards.json'),
    'text': ('json', 'loc_en.json'),
}

headers = {
    'authority': 'api.mtga.untapped.gg',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,pt;q=0.8',
    'if-none-match': '"047066ff947f01e9e609ca4cf0d6c0a6"',
    'origin': 'https://mtga.untapped.gg',
    'referer': 'https://mtga.untapped.gg/',
    'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
}


In [2]:
@lru_cache
def request(keyword, send_headers=True, format=''):
    'Returns JSON from corresponding keyword'
    url_kw, endpoint = endpoints[keyword]
    url = urls[url_kw]
    
    sleep(1) # :)
    if send_headers:
        return requests.get(url+endpoint+format, headers).json()
    else:
        return requests.get(url+endpoint+format).json()

Active

In [3]:
def request_active():
    'Returns format ID and lists of standard legal sets'
    latest = dict()

    # Extracting information from the lastest standard BO1 format
    for format in request('active'):
        if format['event_name'] == 'Ladder':
            latest = format

    return str(latest['id']), latest['legal_sets']

format_id, legal_sets = request_active()
print(format_id, legal_sets)

355 ['MID', 'VOW', 'NEO', 'SNC', 'DMU', 'BRO', 'ONE', 'MOM']


Cards

In [4]:
def request_cards(sets):
    'Returns transformed cards dataframe'
    df = pd.DataFrame(request('cards'))

    # Ony bother with standard legal cards
    df = df[df.set.isin(sets)]

    # Remove duplicates by considering only the latest reprint
    gb = df.groupby('titleId').agg({'grpid':'max'})
    df = df[df.grpid.isin(gb.grpid)]

    return df.set_index('grpid')

raw_card = request_cards(legal_sets)

Text

In [5]:
def request_text():
    'Returns card text dataframe'
    df = pd.DataFrame(request('text')).set_index('id')

    # Collapse columns raw and text, prioritizing raw
    df.loc[~df.raw.isna(), 'text'] = df.raw
    df.drop('raw', axis='columns', inplace=True)

    return df

raw_text = request_text()

Card information

In [6]:
def filter_raw_card(raw_card):
    'Remove undesirable columns, change naming conventions and change index to titleId'
    
    # Columns to ignore and corresponding reasoning
    ignore = [
        'artistCredit',                  # Deemed not useful for analysis     
        'collectorNumber',               # Deemed not useful for analysis
        'collectorMax',                  # Deemed not useful for analysis
        'types',                         # Aggregation from other columns
        'subtypes',                      # Aggregation from other columns
        'colors',                        # Aggregation from other columns
        'colorIdentity',                 # Aggregation from other columns
        'frameColors',                   # Deemed not useful for analysis
        'supertypes',                    # Aggregation from other columns
        'rawFrameDetails',               # Deemed not useful for analysis
        'altDeckLimit',                  # Not standard relevant
        'DigitalReleaseSet',             # Not standard relevant
        'frameDetails',                  # Deemed not useful for analysis
        'abilityIdToLinkedTokenGrpId',   # Not standard relevant
        'linkedFaceType',                # Deemed not useful for analysis
        'linkedFaces',                   # Deemed not useful for analysis
        'usesSideboard',                 # Not standard relevant
        'IsDigitalOnly',                 # Not standard relevant
        'watermark',                     # Deemed not useful for analysis
        'RebalancedCardLink',            # Deemed not useful for analysis
        'altTitleId',                    # Deemed not useful for analysis
        'DefunctRebalancedCardLink',     # Deemed not useful for analysis
        'abilityIdToLinkedConjurations', # Not standard relevant
        'grpid',                         # Obsolete, as titleId is now unique
        'hiddenAbilities',               # Already contained in 'abilities'
    ]

    # Naming conventions
    # raw_card.index.rename('card_id', inplace=True)
    raw_card.rename({
        'set':'set_id',
        'isSecondaryCard': 'is_secondary_card',
        'isToken': 'is_token',
        'IsRebalanced': 'is_rebalanced',
        'artId': 'art_id'},
        axis = 'columns',
        inplace = True
    )

    # Set titleId to index and drop undesirable columns
    return raw_card.reset_index().drop(ignore, axis='columns')

In [7]:
def get_card_dataframe(df):
    "Returns card dataframe"   

    # Card columns mappable to text
    id_to_text = ['titleId', 'flavorId', 'cardTypeTextId', 'subtypeTextId']

    # Additional text_columns columns
    df = df.join(
        df[id_to_text]
        .applymap(lambda x: raw_text.loc[x].values[0], na_action='ignore')
        .rename(columns={column: column[:-2] for column in id_to_text}) # Remove Id
    )

    # Only tracked supertype will be 'legendary'
    df['is_legendary'] = df.cardTypeText.str.contains('Legendary')

    # Replace empty string flavor with NaN
    df['flavor'] = df['flavor'].replace({'': np.nan})

    # Define the columns and order for card dataframe
    order = [
        'titleId',
        'art_id',
        'set_id',
        'title',
        'rarity',
        'power',
        'toughness',
        'flavor',
        'is_legendary',
        'is_token',
        'is_secondary_card',
        'is_rebalanced',
    ]

    return df[order]

In [8]:
def get_card_type_dataframe(df):
    'Returns card type dataframe'
    # Convert 'cardTypeTextId' to text
    df = df.join(
        df.subtypeTextId
        .map(lambda x: raw_text.loc[x].values[0], na_action='ignore')
        .rename('type')
    )
    
    # Split the text and transform into a list of rows, deleting the ones without information
    df = df.type.str.split().explode().dropna()

    # There are special cases of cards not having types such as cards 'Day' and 'Night'
    df = df[~df.isin(['NONE', 'Legendary', 'Basic', 'Token'])]

    return df

In [9]:
def get_card_subtype_dataframe(df):
    "Returns card subtype dataframe"
    # Convert 'cardTypeTextId' to text
    df = df.join(
        df.subtypeTextId
        .map(lambda x: raw_text.loc[x].values[0], na_action='ignore')
        .rename('subtype') 
    )

    # Split the text and transform into a list of rows, deleting the ones without information
    df = df.subtype.str.split().explode().dropna()
    
    return df

In [10]:
def get_card_cost(df):
    "Returns card cost dataframe"

    # Casting color codes in 'castingcost' column
    casting_colors = {
        'Black': r'oB',
        'Blue': r'oU',
        'Green': r'oG',
        'Red': r'oR',
        'White': r'oW',
        'Multicolor': r'\(',
        'X': r'oX'
    }

    # Create a column for each variety
    for color, code in casting_colors.items():
        df[color] = df.castingcost.str.count(code)

    # Special case is colorless that can be any number
    df['Colorless'] = df.castingcost.str.extract('(\d+)')

    # Stack columns into rows and set index to card_id only
    df = (df[['Colorless'] + list(casting_colors.keys())].stack()
                                                         .reset_index()
                                                         .set_index('card_id')
                                                         .rename(columns={'level_1': 'color', 0: 'cost'}))

    df.cost = pd.to_numeric(df.cost) # Convert from string to number

    # Only record costs above zero
    return df[df.cost > 0]

In [11]:
def get_card_ability(df):
    df = (df.abilities
            .dropna()
            .explode()
            .apply(lambda x: x.get('TextId'))
            .map(lambda x: raw_text.loc[x].values[0]))      
    return df

In [12]:
def get_card_information():
    'Returns multiple dataframes containing card information after normalization'
    filtered = filter_raw_card(raw_card)
    card = get_card_dataframe(filtered)

    # Rename 'titleId' to 'card_id' and promote it to index
    for df in [filtered, card]:
        df.set_index('titleId', inplace=True)
        df.index.name = 'card_id'

    card_type = get_card_type_dataframe(filtered)
    card_subtype = get_card_subtype_dataframe(filtered)
    card_cost = get_card_cost(filtered)
    card_ability = get_card_ability(filtered)

    return filtered, card, card_type, card_subtype, card_cost, card_ability

filtered, card, card_type, card_subtype, card_cost, card_ability = get_card_information()

Analytics

In [14]:
def request_analytics(format):
    json = request('analytics', False, format)

    # Separate into data and medatada
    metadata = json['metadata']
    df = pd.json_normalize(json['data']).T

    # Number of games per tier
    games = {tier[0]: metadata['games'][tier]['ALL'] 
             for tier in metadata['games']}

    # Nested list with two levels
    level_1 = ['games', 'wins', 'check', 'copies']
    level_2 = ['copies_1', 'copies_2', 'copies_3', 'copies_4']

    # Unnest the list in stages
    # When we have a new set, sometimes Untapped chooses to mix BO1 and BO3 statistics
    # Thus we make sure to only get BO1 statistics for consistency.
    df[level_1] = pd.DataFrame(df.explode(0)[0].to_list(), index=df.index).iloc[:, :4]
    df[level_2] = pd.DataFrame(df.copies.to_list(), index=df.index)

    # Remove redundant columns and fill NaN with zeros 
    df.drop([0, 'check', 'copies'], axis = 'columns', inplace = True)
    df.fillna(0, inplace = True)

    # Transform dtypes to int, reset index and rename tiers
    df = (df.astype('int64')
            .reset_index()
            .rename(columns={'index':'raw'}))
    
    # Split raw column into multiple
    df[['titleId', 'archetypeId', 'tier']] = df.raw.str.split('.', expand=True)

    # Record only consolidated data by archetype and change index to titleId
    df = (df[df.archetypeId == 'ALL']
            .drop(['raw', 'archetypeId'], axis=1)
            .set_index('titleId'))
    df.index = df.index.astype('int64')
    
    # Time stamp and format
    df.insert(0, 'dt_analytics', pd.Timestamp.today().strftime('%Y-%m-%d'))
#     df.insert(1, 'format', format)
    
    return df, games

analytics, no_games = request_analytics(format_id)
no_games

{'b': 11576, 'g': 40253, 'p': 299962, 's': 27606}

ABT

In [15]:
def abt(format, sets, tier='ALL', full_report=False):
    "Full dataframe for a given query"
    card_information, *_= get_card_information()
    analytics, games = request_analytics(format)

    # Choose either aggregate statistics per tier or single one
    # Use if clause to calculate 'included' statistic
    if tier == 'ALL': # Aggregate games statistics
        agg_dict = {column : 'sum' 
                    if column not in ['dt_analytics', 'format', 'tier']
                    else 'max'
                    for column in analytics.columns}
        analytics = analytics.groupby('titleId').agg(agg_dict)
        analytics.tier = 'ALL'
        analytics['included'] = analytics.games/sum(games.values())
    else:
        analytics = analytics[analytics.tier == tier]
        analytics['included'] = analytics.games/games[tier]

    # Merge analytics and card information
    merged = (analytics.reset_index()
                       .merge(card_information.reset_index(),
                              how='left', 
                              on='titleId')
                       .set_index('titleId'))
    
    # Winrate
    merged['winrate'] = merged.wins/merged.games

    # Quantity
    copies = ['copies_1', 'copies_2', 'copies_3', 'copies_4']
    merged['quantity'] = merged[copies].idxmax(axis=1)
    
    if full_report:
        return merged
    else:
        return merged[['title',
                       'set',
                       'rarity', 
                       'castingcost', 
                       'winrate', 
                       'games', 
                       'included', 
                       'quantity']]
    
df_abt = abt(format_id, legal_sets)

KeyError: 'titleId'